In [4]:
import pandas as pd
import numpy as np

In [5]:
df_sales = pd.read_csv('../data/sales_clean.csv')
df_items = pd.read_csv('../data/items.csv')
df_shops = pd.read_csv('../data/shops.csv')

In [6]:
#agr mensual
df_monthly = df_sales.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
df_monthly.rename(columns={'item_cnt_day': 'item_cnt_month'}, inplace=True)
df_monthly['item_cnt_month'] = df_monthly['item_cnt_month'].clip(0, 20)

In [7]:
#Obtener rango de meses
months = df_sales['date_block_num'].unique()

#Crear la lista con todas las combinaciones (mes, tienda, producto)
matrix = []
for month in months:
    shops = df_sales[df_sales['date_block_num'] == month]['shop_id'].unique()
    items = df_sales[df_sales['date_block_num'] == month]['item_id'].unique()
    matrix += [[month, shop, item] for shop in shops for item in items]

#Convertir a DF
df_matrix = pd.DataFrame(matrix, columns=['date_block_num', 'shop_id', 'item_id'])

In [8]:
#Unir con el dataframe de ventas para añadir la columna item_cnt_month
df_matrix = df_matrix.merge(df_monthly, on=['date_block_num', 'shop_id', 'item_id'], how='left')

#Rellenar los valores que no tenían ventas con 0
df_matrix['item_cnt_month'] = df_matrix['item_cnt_month'].fillna(0).astype(np.float32)

In [10]:
print("Shape matriz final:", df_matrix.shape)
df_matrix.head()

Shape matriz final: (10884554, 4)


,date_block_num,shop_id,item_id,item_cnt_month
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0


In [11]:
df_matrix.to_csv('../data/matriz_base.csv', index=False)